In [7]:
"""
Classification workflow:

0. Set device
1. Load dataset 
2. Turn into data and lables
3. Train test split
4. Format into torch tensors float32 and set device
5. Check shape of tensors
6. Make model class
7. Choose loss and optimizer
8. Training loop 
"""

import torch

device = "cuda" if torch.cuda.is_available() else "cpu"


from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()

data

{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]], shape=(569, 30)),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1,

In [8]:
from sklearn.model_selection import train_test_split

X = data["data"]
y = data["target"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
X_train, X_test = torch.from_numpy(X_train).to(device), torch.from_numpy(X_test).to(device)

X_train, X_test = X_train.type(torch.float32), X_test.type(torch.float32)

y_train, y_test = torch.from_numpy(y_train).to(device), torch.from_numpy(y_test).to(device)

y_train, y_test = y_train.type(torch.float32), y_test.type(torch.float32)

y_train = y_train.view(-1, 1)
y_test = y_test.view(-1, 1)

In [10]:
class BC(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = torch.nn.Linear(in_features=30, out_features=1000)
        self.l2 = torch.nn.Linear(in_features=1000, out_features=1000)
        self.l3 = torch.nn.Linear(in_features=1000, out_features=1)
        self.relu = torch.nn.ReLU()
    
    def forward(self, x):
        hidden_layers = 5
        
        current = self.l1(x)
        for _ in range(hidden_layers):
            current = self.relu(self.l2(current))
        return self.l3(current)

In [11]:
model = BC().to(device)

loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

def acc(logit):
    y_pred = torch.sigmoid(logit)
    y_pred = torch.round(y_pred)
    correct = (y_pred == y_train).sum().item()
    return correct / len(y_train)

In [12]:
epochs = 1000

for epoch in range(epochs):
    model.train

    y_pred = model(X_train) 

    loss = loss_fn(y_pred, y_train)

    accuracy = acc(y_pred)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model.eval
    with torch.no_grad():
        y_test_pred = model(X_test)
        loss_test = loss_fn(y_test_pred, y_test)

    if epoch % 100 == 0:
        print(f"Epoch: {epoch} - Loss: {loss} - Test Loss: {loss_test} - Accuracy: {accuracy}")

Epoch: 0 - Loss: 0.8496070504188538 - Test Loss: 10.742731094360352 - Accuracy: 0.6175824175824176
Epoch: 100 - Loss: 0.2102406769990921 - Test Loss: 0.1643746793270111 - Accuracy: 0.9142857142857143
Epoch: 200 - Loss: 0.22987215220928192 - Test Loss: 0.24672257900238037 - Accuracy: 0.9120879120879121
Epoch: 300 - Loss: 0.16685239970684052 - Test Loss: 0.1138283833861351 - Accuracy: 0.9274725274725275
Epoch: 400 - Loss: 0.14149300754070282 - Test Loss: 0.09469428658485413 - Accuracy: 0.9362637362637363
Epoch: 500 - Loss: 0.15566933155059814 - Test Loss: 0.10738299041986465 - Accuracy: 0.9362637362637363
Epoch: 600 - Loss: 0.14215049147605896 - Test Loss: 0.08084136247634888 - Accuracy: 0.9406593406593406
Epoch: 700 - Loss: 0.19356000423431396 - Test Loss: 0.12103395164012909 - Accuracy: 0.9186813186813186
Epoch: 800 - Loss: 0.1336699277162552 - Test Loss: 0.07390036433935165 - Accuracy: 0.9384615384615385
Epoch: 900 - Loss: 0.11519014835357666 - Test Loss: 0.11317375302314758 - Accurac